# Retention dataset

In [1]:
import pandas as pd
import numpy as np
import pyarrow
from datetime import datetime, date, timedelta
import time

from google.cloud import storage
from google.cloud import bigquery

import sys
import os

sys.path.append(os.path.realpath('../../'))
import data_generator

In [2]:
bigquery_client = bigquery.Client.from_service_account_json('../../credentials/data-analysis-sql-309220-6ce084250abd.json')

In [5]:
params = {
    'dates': {
        'date': {
            'start': '2021-01-01', 
            'end': '2021-03-31'
        }
    }, 
    'categorical': {
        'group': {
            'categories': ['control', 'experimental'],
            'probs': [0.5, 0.5],
            'indices': [1, 1.05]
        },
        'country': {
            'categories': ['UK', 'DE', 'FR', 'IT'], 
            'probs': [0.31, 0.27, 0.23, 0.19], 
            'indices': [0.98, 1.03, 0.99, 1.02]
        }, 
        'platform': {
            'categories': ['android', 'ios', 'web'], 
            'probs': [0.41, 0.29, 0.3], 
            'indices': [0.99, 1.06, 0.94]
        }
    }, 
    'values': {
        'amount': {
            'distribution': 'exponential', 
            'parameters': {
                'scale': 10
            }, 
            'indices': True, 
            'min': 1, 
            'round': 2
        }
    }
}


df = data_generator.create_dataset(rows=10000, params=params, seed=1)
df.head()

,date,group,country,platform,amount
0,2021-01-09,experimental,UK,ios,41.44
1,2021-03-08,control,UK,web,3.97
2,2021-02-18,control,FR,android,31.76
3,2021-03-09,control,FR,android,1.38
4,2021-02-08,experimental,IT,ios,10.67


In [9]:
print(df.sample(frac=0.001).to_markdown(index=False))

| date       | group        | country   | platform   |   amount |
|:-----------|:-------------|:----------|:-----------|---------:|
| 2021-02-15 | control      | DE        | android    |     1.5  |
| 2021-01-25 | experimental | DE        | android    |     7.87 |
| 2021-01-20 | experimental | FR        | ios        |    30.97 |
| 2021-01-30 | experimental | FR        | web        |     2.77 |
| 2021-01-26 | control      | IT        | ios        |     1.7  |
| 2021-01-18 | experimental | DE        | android    |    27.6  |
| 2021-02-09 | experimental | IT        | android    |     1.62 |
| 2021-03-23 | control      | FR        | web        |     3.97 |
| 2021-02-11 | control      | IT        | android    |    16.54 |
| 2021-03-05 | control      | IT        | android    |    13.72 |


### Write the dataset to BigQuery

In [10]:
df.to_csv('payments.csv', index=False)

In [11]:
table_ref = bigquery_client.dataset('synthetic').table('payments')

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1 # ignore the header
job_config.autodetect = True

with open('payments.csv', 'rb') as source_file:
    job = bigquery_client.load_table_from_file(
        source_file, table_ref, job_config=job_config
    )

job.result()

In [12]:
%%bash
rm payments.csv